In [1]:
from MFEAcode import *
from GNBG import GNBG
from tqdm import tqdm

In [2]:
class EMEBI:
    def __init__(self, prob, BASE_POPSZ, BASE_rmp, gen_length, MAX_FES, update_rate, learning, dynamic_pop, save_pop = False):
        self.prob = prob
        self.base_popsize = 100
        self.max_popsize = 100
        self.min_popsize = 20
        self.inds_tasks = [self.base_popsize] * len(prob)
        self.base_rmp = BASE_rmp
        self.gen_length = gen_length
        self.MAX_FES = MAX_FES
        self.delta = None 
        self.s_rmp = None
        self.rmp = np.full((len(prob), len(prob)), BASE_rmp)
        self.update_rate = update_rate
        self.learningPhase = [LearningPhase(is_start = True, prob = prob) for t in range(len(prob))]
        self.learning = learning
        self.dynamic_pop = dynamic_pop
        self.save_pop = save_pop
        self.log = [ [] for _ in range(len(prob))]
    def run(self, checkpoint = None):
        np.fill_diagonal(self.rmp, 0)
        
        Parameter.FEl = np.zeros((len(self.prob)))
        Parameter.FEs = 0
        best = []
        if checkpoint is None:
            pop = Population(sum(self.inds_tasks), self.gen_length, self.prob)
            pop.init(opposition=True)
            pop.update_scalar_fitness()
            D0, _, _ = pop.calculateD()
        if checkpoint is not None:
            pop = checkpoint 
            pop.update_scalar_fitness()
            D0, _, _ = pop.calculateD()
        this_pop_result = pop.get_result()
        for i in range(len(self.prob)):
            self.log[i].append([Parameter.FEl[i], this_pop_result[i]])
        progress_bar = tqdm(total=self.MAX_FES, unit = 'FEs', unit_scale = True)
        while True:
            old_fes = Parameter.FEs
                 #########################  PHASE 1 - EMEBI ############################
            # self.delta = [[[] for _ in range(len(self.prob))] for _ in range(len(self.prob))]
            # self.s_rmp = [[[] for _ in range(len(self.prob))] for _ in range(len(self.prob))]
            # offsprings = self.reproduction(pop, sum(self.inds_tasks))
            # pop.pop.extend(offsprings)
            # if self.dynamic_pop:
            #     self.inds_tasks = [int(
            #         int(max((self.min_popsize - self.max_popsize) * (Parameter.FEs/self.MAX_FES) + self.max_popsize, self.min_popsize))
            #     )] * len(self.prob)
            # pop.selection_EMEBI(self.inds_tasks)  #each skill-factor choose some best
            # # pop.update_scalar_fitness()                 # choose best of the whole pop using scalar-fitness
            # # pop.selection(sum(self.inds_tasks))
                  
            # self.updateRMP(self.update_rate)
               #########################  PHASE 1 - MFEA2 ############################
            offsprings = pop.reproduction_MFEA_2(sum(self.inds_tasks))
            pop.pop.extend(offsprings)
            if self.dynamic_pop:
                self.inds_tasks = [int(
                    int(max((self.min_popsize - self.max_popsize) * (Parameter.FEs/self.MAX_FES) + self.max_popsize, self.min_popsize))
                )] * len(self.prob)
            pop.selection(sum(self.inds_tasks))
            if self.learning:
                self.phaseTwo(D0, pop)
            new_fes = Parameter.FEs
            if Parameter.FEs <= self.MAX_FES:
                this_pop_result = pop.get_result()
                for i in range(len(self.prob)):
                    self.log[i].append([Parameter.FEl[i], this_pop_result[i]])
                result = "Current fitness: "
                for i in range(len(self.prob)):
                    result += f"{i+1}: {this_pop_result[i]} "
                progress_bar.set_description(result)
                progress_bar.update(new_fes - old_fes)
            else:
                break
        return self.log, pop
            


    def reproduction(self, pop, SIZE):
        sub_size = int(SIZE/len(self.prob))
        offs = []
        population = pop.pop
        counter = np.zeros((len(self.prob)))
        terminateCondition = False
        while not terminateCondition:
            idx_1, idx_2 = np.random.choice(len(population), 2)
            ind1 = population[idx_1]
            ind2 = population[idx_2]
            t1 = ind1.skill_factor
            t2 = ind2.skill_factor
            if counter[t1-1] >= sub_size and counter[t2-1] >= sub_size:
                continue 
            rmpValue = np.random.normal(loc = max(self.rmp[t1-1][t2-1], self.rmp[t2-1][t1-1]), scale = 0.1)
            if t1 == t2:
                off1, off2 = pop.crossover(ind1, ind2)
                off1.skill_factor = t1
                off2.skill_factor = t2
                off1.cal_fitness(self.prob)
                off2.cal_fitness(self.prob)
                offs.extend([off1, off2])
                counter[t1-1] += 2
            elif random.random() < rmpValue:
                this_offs = pop.crossover(ind1, ind2)
                for off in this_offs:
                    if counter[t1-1] < sub_size and random.random() < self.rmp[t1-1][t2-1] / (self.rmp[t1-1][t2-1] + self.rmp[t2-1][t1-1]):
                        off.skill_factor = t1
                        off.cal_fitness(self.prob)
                      
                        offs.append(off)
                        counter[t1-1] += 1
                        if ind1.fitness[t1-1] > off.fitness[t1-1]:
                            self.delta[t1-1][t2-1].append(ind1.fitness[t1-1] - off.fitness[t1-1])
                            self.s_rmp[t1-1][t2-1].append(rmpValue)
                    elif counter[t2-1]< sub_size:
                        off.skill_factor = t2 
                        off.cal_fitness(self.prob)
                    
                     
                        offs.append(off)
                        counter[t2-1] +=1
                        if ind2.fitness[t2-1] > off.fitness[t2-1]:
                            self.delta[t2-1][t1-1].append(ind2.fitness[t2-1] - off.fitness[t2-1])
                            self.s_rmp[t2-1][t1-1].append(rmpValue)
            else:
                if counter[t1 - 1] < sub_size:
                            sf_list = np.array( [pop.skill_factor for pop in population] )
                           
                            idx_same_sf = np.where(sf_list == t1)[0]
                   
                            random_idx = np.random.choice(idx_same_sf)
                            ind11 = population[random_idx]
                            assert ind11.skill_factor == t1
                            off1, _ = pop.crossover(ind1, ind11)
                            off1.skill_factor = t1
                            off1.cal_fitness(self.prob)
                            offs.append(off1)
                            counter[t1-1] += 1
                if counter[t2 - 1] < sub_size:
                            sf_list = np.array( [pop.skill_factor for pop in population] )
                            idx_same_sf = np.where(sf_list == t2)[0]
                            random_idx = np.random.choice(idx_same_sf)
                            ind22 = population[random_idx]
                            assert ind22.skill_factor == t2
                            _, off2 = pop.crossover(ind2, ind22)
                            off2.skill_factor = t2
                            off2.cal_fitness(self.prob)
                            offs.append(off2)
                            counter[t2-1] += 1
            terminateCondition = sum(counter >= sub_size) == len(self.prob)
        return offs
    def updateRMP(self, update_rate):
            for i in range(len(self.prob)):
                for j in range(len(self.prob)):
                    if i==j:
                        continue 
                    if len(self.delta[i][j]) > 0:
                        self.rmp[i][j] += update_rate * Lehmer_mean(self.delta[i][j], self.s_rmp[i][j])
                    else:
                        self.rmp[i][j] = (1-update_rate)*self.rmp[i][j]
                    self.rmp[i][j] = max(0.1, min(1, self.rmp[i][j]))
    def phaseTwo(self, D0, pop):
        D, maxFit, minFit = pop.calculateD()
        maxDelta = maxFit - minFit + 1e-99
        sigma = np.where(D > D0 , 0 , 1 - D/D0)
        newPop = []
        subpops = pop.get_subpops()
        for i in range(len(self.prob)):
            nextPop = self.learningPhase[i].evolve(subpops[i], sigma[i], maxDelta[i], divisionRate =0.3)
            newPop.extend(nextPop)
            self.learningPhase[i].start = False
        pop.pop = newPop
        

def Lehmer_mean(delta, s_rmp):
        delta = np.array(delta)
        s_rmp = np.array(s_rmp)
        sum_delta = sum(delta)
        tmp = (delta/sum_delta) * s_rmp
        meanS = sum(tmp * s_rmp)
        return meanS/sum(tmp)



In [3]:
class SubPop():
    def __init__(self, pool, prob, searcher=None):
        self.list_tasks = prob
        self.pool = pool      #expect a list of Individuals with the same skill_factor, that is self.skill_factor
        self.searcher = searcher 
        self.fitness_improv = 0
        self.consume_fes = 0
        self.mem_cr = [0.5] * LearningPhase.H
        self.mem_f = [0.5] * LearningPhase.H
        self.dim = len(pool[0].genes)
        self.len_tasks = len(pool[0].fitness)
        self.skill_factor = pool[0].skill_factor
        # self.task = prob[self.skill_factor - 1]
        self.scale = 0.1
    def sort_fit(self):
        self.pool.sort(key=lambda ind: ind.fitness[self.skill_factor - 1])
    def mutation_ind(self, parent):
        p = parent.genes 
        mp = float(1. / p.shape[0])
        u = np.random.uniform(size=[p.shape[0]])
        r = np.random.uniform(size=[p.shape[0]])
        tmp = np.copy(p)
        for i in range(p.shape[0]):
            if r[i] < mp:
                if u[i] < 0.5:
                    delta = (2*u[i]) ** (1/(1+Parameter.mum)) - 1
                    tmp[i] = p[i] + delta * p[i]
                else:
                    delta = 1 - (2 * (1 - u[i])) ** (1/(1+Parameter.mum))
                    tmp[i] = p[i] + delta * (1 - p[i])
        tmp = np.clip(tmp, 0, 1)
        ind = Individual(self.dim, self.len_tasks)
        ind.genes = tmp
        ind.skill_factor = parent.skill_factor
        ind.cal_fitness(self.list_tasks)
        return ind
    def pbest_ind(self, ind, cr, f, best_rate):
        assert len(self.pool) > 0
        best_size = max (int (len(self.pool) * best_rate), 1)
        best = self.pool[:best_size]
        pbest = best[random.randint(0, len(best) - 1)]
    
        rand_idx = np.random.choice(len(self.pool), 2, replace=False)
        ind_ran1, ind_ran2 = self.pool[rand_idx[0]], self.pool[rand_idx[1]]
        u = (np.random.uniform(0, 1, size=(len(ind.genes)) ) <= cr)
        u[np.random.choice(self.dim)] = True

        new_genes = np.where(u, 
            pbest.genes + f * (ind_ran1.genes - ind_ran2.genes),
            ind.genes
        )
        new_genes = np.clip(new_genes, 0, 1)
        new_ind = Individual(self.dim, self.len_tasks)
        new_ind.genes = new_genes
        new_ind.skill_factor = ind.skill_factor
        new_ind.cal_fitness(self.list_tasks)
        return new_ind
    def search(self, mem_cr, mem_f,  sigma, max_delta, best_rate):
        s_f = []
        s_cr = []
        diff_f = []
        accumulate_diff = 0
        self.sort_fit()
        new_pool = []
        start_FE = Parameter.FEs
        for idx, ind in enumerate(self.pool):
            r = random.randint(0, LearningPhase.H - 1)
            if(idx==0):
                new_pool.append(ind)
                continue
            print("idx=0 still run ???")
            while True:
                cr = min(np.random.normal(mem_cr[r], 0.1), 1)
                sub_f = np.random.standard_cauchy()
                f = min(mem_f[r] + 0.1 * sub_f, 1)
                if(cr>=0 and f>=0):
                    break
            if self.searcher == "Mutation":
                new_ind = self.mutation_ind(ind)
            if self.searcher == "PBest":
                new_ind = self.pbest_ind(ind, cr, f, best_rate)
            diff = ind.fitness[ind.skill_factor-1] - new_ind.fitness[new_ind.skill_factor-1]
            
            if diff > 0:
                accumulate_diff += diff
                new_pool.append(new_ind)
                if self.searcher == "PBest":
                    diff_f.append(diff)
                    s_cr.append(cr)
                    s_f.append(f)
            else:
                new_pool.append(ind)
                # survival_rate = sigma * np.exp(diff/max_delta)
                # if random.random() < survival_rate:
                #     new_pool.append(new_ind)
                # else:
                #     new_pool.append(ind)
        end_FE = Parameter.FEs
        effective = accumulate_diff / (end_FE - start_FE)
        assert end_FE - start_FE > 0
        assert len(new_pool) == len(self.pool)
        if self.searcher == "PBest":
            return [new_pool, diff_f, s_cr, s_f, effective]
        else:
            return [new_pool, effective]
    
        
        
        
        

In [4]:
class LearningPhase():
    M = 2   #number of operators
    H = 10   #F, CR list length
    def __init__(self, is_start, prob) -> None:
        # self.list_tasks = list_tasks
        # self.num_task = len(list_tasks)
        # self.dim = dim
        self.list_tasks = prob
        self.effective = [0] * LearningPhase.M
        self.mem_cr = [0.5] * LearningPhase.H
        self.mem_f = [0.5] * LearningPhase.H
        self.s_cr = []
        self.s_f = []
        self.diff_f = []
        self.mem_pos = 0
        self.scale = 0.1
        self.start = is_start
        self.searcher = ["PBest", "Mutation"]
    def evolve(self, subpop,  sigma: float, max_delta: float, divisionRate: float,) :
        # divide subPop wrt divisionRate
        new_pool = []
        random.shuffle(subpop)
        subpop1 = subpop[int(len(subpop)*divisionRate):]   #major
        subpop2 = subpop[:int(len(subpop)*divisionRate)]
        SubPop1 = SubPop(subpop1, self.list_tasks)
        SubPop2 = SubPop(subpop2, self.list_tasks)
        opcode = 0
        if self.start:
            opcode = random.randint(0,1)
            SubPop1.searcher = self.searcher[opcode]
            SubPop2.searcher = self.searcher[1 - opcode]
        else:
            self.updateMemory()
            opcode = np.argmax(np.array(self.effective))
            SubPop1.searcher = self.searcher[opcode]
            SubPop2.searcher = self.searcher[1 - opcode]
        pool1 = SubPop1.search(self.mem_cr, self.mem_f,  sigma, max_delta, best_rate = 0.1)
        pool2 = SubPop2.search(self.mem_cr, self.mem_f,  sigma, max_delta , best_rate = 0.1)
        
        for pooli in [pool1, pool2]:
            if(len(pooli)==2):
                new_pool.extend(pooli[0])
                self.effective[opcode] = pooli[1]
            if(len(pooli)==5):
                new_pool.extend(pooli[0])
                self.diff_f, self.s_cr, self.s_f , self.effective[1-opcode]= pooli[1], pooli[2], pooli[3], pooli[4]
        return new_pool
        
    # def evolve(self, subPop, sigma: float, max_delta: float) :
    #     eval_k = 0
    #     pool = []
        
    #     self.gen += 1
    #     if self.gen > 1:
    #         self.best_opcode = self.__class__.updateOperator(sum_improve = self.sum_improv, 
    #                                                          consume_fes = self.consume_fes, 
    #                                                          M = LearningPhase.M)

    #         # self.sum_improv = np.zeros((LearningPhase.M))
    #         # self.consume_fes = np.ones((LearningPhase.M))
    #         self.sum_improv = [0.0] * LearningPhase.M
    #         self.consume_fes = [1.0] * LearningPhase.M

        # self.updateMemory()
        
        # pbest_size = int( 0.1 * len(subPop) )
        # subPop.sort(key = lambda ind : ind.fitness[ind.skill_factor-1])
        # idx = subPop[:pbest_size]
        # pbest = [subPop[i] for i in idx]

        # for ind in subPop:
        #     r = random.randint(0, LearningPhase.H - 1)
        #     cr = np.random.normal(self.mem_cr[r], 0.1)
        #     sub_f = np.random.standard_cauchy()
        #     f = self.mem_f[r] + 0.1 * sub_f
        #     # f = np.random.cauchy(self.mem_f[r], 0.1)
                        
        #     opcode = random.randint(0, LearningPhase.M)
        #     if opcode == LearningPhase.M:
        #         opcode = self.best_opcode
            
        #     if opcode == 0:
        #         child = self.searcher[opcode](ind, subPop, pbest, cr, f)
        #     elif opcode == 1:
        #         child = self.searcher[opcode](ind, return_newInd=True)

        #     child.skill_factor = ind.skill_factor
        #     child.cal_fitness(self.list_tasks)
            
            
        #     diff = ind.fitness[ind.skill_factor-1] - child.fitness[child.skill_factor-1]
        #     if diff > 0:
        #         print("YAY !")
        #         survival = child

        #         self.sum_improv[opcode] += diff

        #         if opcode == 0:
        #             self.diff_f.append(diff)
        #             self.s_cr.append(cr)
        #             self.s_f.append(f)
                
        #     elif diff == 0 or random.random() <= sigma * np.exp(diff/max_delta):
        #         survival = child
        #     else:
        #         survival = ind
            
        #     pool.append(survival)
        
        # return pool
    


    def updateMemory(self):
        if len(self.s_cr) > 0:
        
            self.mem_cr[self.mem_pos] = self.updateMemoryCR(self.diff_f, self.s_cr)
            self.mem_f[self.mem_pos] = self.updateMemoryF(self.diff_f, self.s_f)
            
            self.mem_pos = (self.mem_pos + 1) % LearningPhase.H

            self.s_cr = []
            self.s_f = []
            self.diff_f = []
        else:
            pass

    def updateMemoryCR(self, diff_f, s_cr) -> float:
        diff_f = np.array(diff_f)
        s_cr = np.array(s_cr)

        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tmp_sum_cr = sum(weight * s_cr)
        return tmp_sum_cr
        # mem_cr = sum(weight * s_cr * s_cr)
        
        # if tmp_sum_cr == 0 or mem_cr == -1:
        #     raise ValueError("Oh no")
        #     return -1
        # else:
        #     return mem_cr/tmp_sum_cr
        
    def updateMemoryF(self, diff_f, s_f) -> float:
        diff_f = np.array(diff_f)
        s_f = np.array(s_f)

        sum_diff = sum(diff_f)
        weight = diff_f/sum_diff
        tmp_sum_f = sum(weight * s_f)
        return sum(weight * (s_f ** 2)) / tmp_sum_f

  
    # def updateOperator(sum_improve, consume_fes, M: int) -> int:
    #     sum_improve = np.array(sum_improve)
    #     consume_fes = np.array(consume_fes)
    #     eta = sum_improve / consume_fes
    #     best_rate = max(eta)
    #     best_op = np.argmax(eta)
    #     if best_rate > 0:
    #         return best_op
    #     else:
    #         return random.randint(0, M - 1)

In [11]:


my_list = [2, 3, 4, 5]
random.shuffle(my_list)
print(my_list)



[3, 2, 4, 5]


### Normal run

In [6]:
Algo = EMEBI(prob= function_list, BASE_POPSZ=100, BASE_rmp=0.3, gen_length=30, MAX_FES=40000, update_rate=0.06, learning=True, dynamic_pop=True)

In [7]:
import copy
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
class Execute:
    def __init__(self, algo, problems, num_run):
        self.problems = problems
        self.bestValue = [prob.opt_value for prob in problems]
        self.algo = algo
        self.num_run = num_run
        self.attempt = np.zeros((num_run, len(self.algo), len(self.problems)))
        self.logs = []
    def run(self):
        for i in range(self.num_run):
            this_algo = copy.deepcopy(self.algo)
            tmp_log = []
            for idx, algo in enumerate(this_algo):
                algo.run()
                self.attempt[i][idx] = [self.bestValue[t] - algo.log[t][-1][-1] for t in range(len(self.problems))]
                tmp_log.append(algo.log)
            self.logs.append(tmp_log)
    def convergence_plot(self, Task):
        for i in range(len(self.algo)):
            LOG = self.logs[0][i][Task - 1]
            FEs = [logg[0] for logg in LOG]
            fitness = [logg[1] for logg in LOG]
            plt.plot(FEs, fitness, label= self.algo[i].name)
        # plt.yscale("log")
        plt.legend()
        plt.show()
    def statistic(self):
        for idx, algo in enumerate(self.algo):
            print(self.algo[i].name + ": " + "\n")
            attempt = self.attempt[:, idx, :]
            mean_values = attempt.mean(axis=0)
            std_values = attempt.std(axis=0)
            for col_idx, (mean, std) in enumerate(zip(mean_values, std_values)):
                print(f"Task {col_idx+1}:")
                print(f"Mean: {mean}")
                print(f"Std: {std}")
                print("-----------------------")
            print("----------------------------\n")


In [8]:
exe = Execute(algo=[Algo], problems=problems, num_run=2)
exe.run()

Current fitness: 1: 56548.779524093305 2: -4040.362459224014 :   1%|          | 394/40.0k [00:00<00:19, 2.02kFEs/s]

idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???


Current fitness: 1: 47278.15821541838 2: -4040.362459224014 :   2%|▏         | 783/40.0k [00:00<00:19, 1.97kFEs/s] 

idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???
idx=0 still run ???


KeyboardInterrupt: 

Current fitness: 1: 47278.15821541838 2: -4040.362459224014 :   2%|▏         | 783/40.0k [00:20<00:19, 1.97kFEs/s]

In [ ]:
exe.statistic()

Algorithm 1: 

Column 1:
Mean: -682.1000910462726
Std: 3.837864460365381e-05
-----------------------
Column 2:
Mean: 0.0
Std: 0.0
-----------------------
----------------------------

